In [2]:
### Different approach - batch size learning 
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)
from client import Client
from fedavg import Fedavg
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils import shuffle
from supported_modles import Supported_modles
import random
import utils

In [3]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [4]:
selected_model = Supported_modles.logistic_regression

In [5]:
clients = utils.set_data()
f1_before = []
for client in clients:
    client.split_data()
    # client.prep_data()
    client.train_model(selected_model)
    print(client.y.mean())
    f1_before.append(client.f1)

Creating node1.
Creating node2.
Creating node3.
Creating node4.
Creating node5.
0.5471641791044776
0.5564179104477612
0.5491044776119403
0.5505970149253732
0.5554738671300629


In [6]:
f1_before

[0.8685314685314685,
 0.8647178789938818,
 0.8725701943844493,
 0.8846880907372401,
 0.8766091814427982]

In [7]:
selected_model = Supported_modles.SGD_classifier

In [8]:
model = clients[0].model
for client in clients:
    client.split_data()
    client.init_empty_model(selected_model,model)

In [21]:
fedavg = Fedavg("global", 0.1)
fedavg.init_global_model(selected_model, clients[0].model, 57)

[Errno 98] Address already in use
Waitiing for a Connection..


In [28]:
number_of_rounds = 40
batch_size = 0.2
epochs = 20
# prep = StandardScaler() 

for _ in range(number_of_rounds):
    print(f'Starting new round!')

    applicable_clients = random.sample((clients), random.randint(1, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0
    

    for client in applicable_clients:
        # print(f'Client name: {client.name}')

        X = client.x
        y = client.y

        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=batch_size, stratify=y, random_state=random.randint(0,10))
       
        # X_train = prep.fit_transform(X_train)

        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        fedavg.load_global_model(client.model, selected_model)
        fedavg.train_local_agent(X_train, y_train, client.model, epochs, sample_weights, selected_model)
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
    # round_weights = weights
    fedavg.update_global_model(applicable_models, round_weights, selected_model)

Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!


In [29]:
f1_fedavg = []
for client in clients:
    # x = prep.transform(client.x_test)
    y_hat = fedavg.model.predict(client.x_test)
    f1_fedavg.append(f1_score(client.y_test,y_hat))
print(f1_fedavg)

[0.8900642236494144, 0.9005275056518462, 0.8902301018483592, 0.8873403019744484, 0.8908534512070725]


In [30]:
i = 0
for client in clients:
    print(f'Difference {f1_fedavg[i]-f1_before[i]}')
    i += 1

Difference 0.021532755117945945
Difference 0.0358096266579645
Difference 0.017659907463909885
Difference 0.0026522112372082685
Difference 0.014244269764274287
